In [1]:
# импорт основных библиотек
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from sklearn import model_selection, metrics, compose, ensemble, preprocessing,  linear_model, svm, neighbors
import xgboost as xgb
import seaborn as sns
import scipy as sc
plt.style.use('ggplot')  # стиль для графиков

# Обзор датасета

In [4]:
# подгружаем данные
data=pd.read_csv('./src/data.csv')
data

,PriceNoTax,YearNum,Month_ID,BrandR1,SeazonClass,ConstrGroup,ExpositionType,CityType,OperatorR1,OperatorR2,BudgetClass,GRP,Latitude,Longitude,Population,Seller_Cat,Dis
0,17000.0,3,2,1.0,1,2,0,2,0,1,3,6.105542,55.457919,65.335799,312364.0,224,0.973449
1,17000.0,3,4,1.0,1,2,0,2,0,1,2,6.105542,55.457919,65.335799,312364.0,224,1.006239
2,21000.0,1,1,3.0,0,1,1,4,0,2,3,8.170000,55.159627,61.381623,1196680.0,268,1.136000
3,27000.0,3,4,1.0,1,2,0,2,0,1,2,3.750000,54.563482,36.298974,332039.0,209,1.131999
4,25000.0,2,8,1.0,1,2,1,4,2,4,3,2.303154,56.791160,60.572966,1493749.0,115,0.999160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13761,24000.0,1,9,1.0,2,2,0,3,2,3,1,3.290000,57.152781,65.620878,807271.0,120,1.098612
13762,18250.0,2,8,1.0,1,2,0,4,2,2,3,2.037000,57.992740,56.180762,1055397.0,222,0.996210
13763,10000.0,3,3,0.0,1,0,2,3,0,2,1,7.980000,57.143039,65.586761,807271.0,112,1.115142
13764,28000.0,2,8,1.0,1,2,0,4,0,1,3,2.434286,56.804553,60.650616,1493749.0,208,1.029619


**Описание полей**
- **PriceNoTax**: прайсовая цена конструкции
- **YearNum**: код года размещения
- **Month_ID**: код месяца размещения
- **BrandR1**: Категория бренда клиента:
 * 0: Глобальный
 * 1: Федеральный
 * 2: Региональный
 * 3: Локальный
- **SeazonClass**: 
 * 0: Низкий (январь, февраль, июнь, июль)
 * 1: Средний (март, апрель, май, август, сентябрь)
 * 2: Высокий (октябрь, ноябрь, декабрь)
- **ConstrGroup**: площадь конструкции:
 * 0: до 5 кв.м
 * 1: 5-15 кв.м
 * 2: 15-20 кв.м
 * 3: 20-100 кв.м
 * 4:более 100 кв.м
- **ExpositionType**: Тип конструкции
 * 0: Статика
 * 1: Тривижн
 * 2: Скроллер
 * 3: Диджитал
- **CityType**: Категория города размещения, в зависимости от населения
 * 0: более 4млн
 * 1: от 990 тыс
 * 2: от 440 тыс
 * 3: от 90 тыс
 * 4: от 45 тыс
 * 5: менее 45 тыс
- **OperatorR1**: география присутствия селлера
 * 0: Локальный
 * 1: Региональный
 * 2: Федеральный
- **OperatorR2**: масштаб селлера - количество конструкции в собственности
 * 0: до 100 конструкций
 * 1: 100-900 констр
 * 2: 900-4900 констр
 * 3: 4900 - 9900 констр
 * 4: более 9900 констр
- **Budget**: категория клиента в зависимости от  размеру рекламного бюджета
- **Latitude**: координаты конструкции (широта)
- **Longitude**: координаты конструкции (долгота)
- **Population**: население города размещения 
- **Seller_Cat**: закодированный селлер конструкции (мелкие селлеры объединены в одну группу 275) 
- **Dis**: целевая переменная: размер дисконта/надбавки к прайсовой цене

**Пару слов о препроцессинге**

Перед выгрузкой датасет был предварительно обработан, препроцесинг включал в себя:
- удаление дубликатов
- сброс NaN
- удаление выбросов: убрано порядка 1% записей, лежащих вне интервала [квантиль 0.25 - 1.5$*$IQR ; квантиль 0.75 + 1.5$*$IQR]
- обработка конфиденциальных данных: бюджеты клиентов и селлеры были закодированы, целевая переменная прошла трансформацию: сдвиг + мат. функция


# EDA

для предварительного анализа данных воспользуется таким прекрасным инструментом как pandas-profiling 

In [ ]:
# установка версии 3.5.0 (последняя на момент написания)
#!pip install pandas-profiling==3.5.0

In [8]:
import pandas_profiling

In [ ]:
import pandas_profiling
# формируем отчет (требователен к ресурсам, может выбросить ошибку, тогда можно сразу перейти к выводам)
profile = pandas_profiling.ProfileReport(data)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Главное, что можно отметить в отчете:
- большинство переменных категориальные, понадобится кодировка
- если смотреть на корреляцию признаков, то кроме очевидных корреляций, вроде сезонности с номером месяца или прайсовой цены с размером конструкции, с которыми особо ничего не сделать, есть сильная корреляция между признаками **OperatorR1** и **OperatorR2**, а также между признаками **CityType** и **Population**. Для первого запуска модели оставим все как есть, но отбор признаков начнем с этой четверки.
- распределения признаков GRP и PriceNoTax, являются степенными, имеют длинные хвосты, распределение признака Population также имеет тяжелый хвост с небольшим пиком в самом конце, целесообразно прологарифмирвать эти признаки, чтобы сгладить распределение

Категориальные признаки разбиваются на 3 группы:
- номинальные (**Month_ID**, **YearNum**, **ExpositionType**) их кодируем с помощью OneHotEncoding
- high-cardinality - с большим количеством уников (**Seller_Cat**), его закодируем с помощью LeaveOneOut
- ordinal (порядковые) все остальные (столбцы с 4 по 11 включительно за минусом **ExpositionType**), их оставим как есть, они уже закодированы порядковыми номерам

# Подготовка данных

In [13]:
# перемешиваем датасет
data =data.sample (frac=1 )

In [14]:
# разделим датасет на признаки и target
y= data.loc[:,'Dis']
X= data.drop ('Dis' , axis=1)

In [15]:
# для кодирования используем библиотеку category_encoders
#!pip install category_encoders
from category_encoders.one_hot import OneHotEncoder
from category_encoders.count import CountEncoder

ohe = OneHotEncoder (cols=['YearNum' , 'Month_ID' , 'ExpositionType'], use_cat_names=True)
X = ohe.fit_transform(X)

# используем частотное кодирование для категории Seller_Cat
coe = CountEncoder (cols=['Seller_Cat'] )
X = coe.fit_transform(X ,y)

In [16]:
# Логарифмирование: определяем функцию трансформации - log(1+x) - и перезаписываем соответствующие столбцы датафрейма
tr = preprocessing.FunctionTransformer(np.log1p, validate=True)
col_tr = compose.make_column_transformer ((tr , ['PriceNoTax' , 'GRP' , 'Population']) )
X[['PriceNoTax' , 'GRP' , 'Population']] = col_tr.fit_transform(X)

In [22]:
#проверяем как прошло кодирование и трансформация
X.describe()

,PriceNoTax,YearNum_1.0,YearNum_2.0,YearNum_3.0,Month_ID_9.0,Month_ID_6.0,Month_ID_1.0,Month_ID_11.0,Month_ID_5.0,Month_ID_3.0,...,ExpositionType_3.0,CityType,OperatorR1,OperatorR2,BudgetClass,GRP,Latitude,Longitude,Population,Seller_Cat
count,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,...,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000,13766.000000
mean,9.896364,0.572715,0.271466,0.155819,0.086154,0.076493,0.107148,0.091748,0.063853,0.084120,...,0.002325,3.042133,0.464550,1.829217,3.371713,1.613245,55.287535,60.811144,13.296133,513.143978
std,0.630720,0.494702,0.444732,0.362697,0.280602,0.265795,0.309313,0.288680,0.244500,0.277578,...,0.048159,1.068370,0.831377,1.087775,1.575211,0.660219,3.744605,18.236020,1.079244,551.178932
min,6.216606,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,42.778389,19.961108,8.989070,1.000000
25%,9.510519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.000000,0.000000,1.000000,2.000000,1.147402,54.728158,50.210082,12.849591,106.000000
50%,9.947552,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,0.000000,2.000000,3.000000,1.646245,55.726300,60.576386,13.357265,284.000000
75%,10.275086,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.000000,0.000000,2.000000,5.000000,1.950187,56.852554,65.578962,13.995062,764.000000
max,13.910822,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,5.000000,2.000000,4.000000,5.000000,4.779208,69.359311,158.654575,16.350419,1782.000000


# Тестирование моделей

In [23]:
# используем RandomForest в качестве baseline, получаем оценку R2 на кросс-валидации 
cvs_RF = model_selection.cross_val_score (ensemble.RandomForestRegressor (), X, y,  cv=5, scoring='r2')
print ('RandomForest, R2 по фолдам ', cvs_RF)
print (f'RandomForest, R2 среднее = {cvs_RF.mean():.3}')

RandomForest, R2 по фолдам  [0.89001917 0.8752196  0.87177171 0.87806893 0.89261617]
RandomForest, R2 среднее = 0.882


In [24]:
# определим функцию, которая будет оценивать пользовательскую метрику (доля предсказаний превышающих допустимый
# интервал отклонения в 5 %)
def custom_loss (y_true, y_pred):
    diff=list(y_true-y_pred)
    list_d = [x for x in diff if x <-0.05 or x > 0.05]
    return len(list_d)/len(diff)*100
# и скорер, который можно использовать в кросс-валидации и оптимизации
custom_scorer = metrics.make_scorer(custom_loss, greater_is_better=False)

In [25]:
# получим оценку custom-loss на кросс-валидации 
cvs_RFC = model_selection.cross_val_score (ensemble.RandomForestRegressor (), X, y,  cv=5, scoring=custom_scorer)
print ('RandomForest, Custom_scor по фолдам ', np.negative (cvs_RFC) )
print (f'RandomForest, Custom_scor, среднее =  {np.negative (cvs_RFC.mean()):.3}')

RandomForest, Custom_scor по фолдам  [5.33769063 5.44860153 5.15800944 5.37595351 4.79476934]
RandomForest, Custom_scor, среднее =  5.22


Чуть-чуть не дотягивает до нужного порога, посмотрим в сторону других моделей

In [26]:
# посмотрим насколько лучше отработает текущий стандарт ML обучения - xgboost
cvs = model_selection.cross_val_score (xgb.XGBRegressor ( ), X, y,  cv=5, scoring='r2')
print ('Xgboost, R2 по фолдам ', cvs)
print (f'Xgboost, R2 среднее = {cvs.mean():.3}')

Xgboost, R2 по фолдам  [0.84528461 0.83736369 0.83491356 0.83819833 0.851123  ]
Xgboost, R2 среднее = 0.841


R2 похуже, но это с дефолтными гиперпараметрами, пробуем подобрать параметры получше

In [27]:
# фиксируем learning rate, перебираем количество итераций и глубину деревьев 
params = {'n_estimators' : [500, 750, 1000, 1250] ,    'max_depth' :[5, 8, 11, 14, 17] }
gs= model_selection.GridSearchCV (xgb.XGBRegressor (learning_rate=0.03 ) , param_grid=params, scoring = 'r2' )

In [53]:
#gs.fit (X, y)

In [54]:
#gs.best_params_

In [28]:
# используем найденные параметры
cvs_XB = model_selection.cross_val_score (xgb.XGBRegressor ( learning_rate=0.03, max_depth=11, n_estimators=1250, objective='reg:squarederror' ),
                                       X, y,  cv=5, scoring='r2')
print ('Xgboost, R2 по фолдам ', cvs_XB)
print (f'Xgboost, R2 среднее = {cvs_XB.mean():.3}')

Xgboost, R2 по фолдам  [0.9025651  0.88594433 0.88213574 0.88061359 0.90365231]
Xgboost, R2 среднее = 0.891


In [29]:
# теперь custom-loss на кросс-валидации 
cvs_XBC = model_selection.cross_val_score (xgb.XGBRegressor ( learning_rate=0.03, max_depth=11, n_estimators=1250 , objective='reg:squarederror'),
                                       X, y,  cv=5, scoring=custom_scorer)
print ('Xgboost, Custom_scor по фолдам ', np.negative (cvs_XBC) )
print (f'Xgboost, Custom_scor, среднее =  {np.negative (cvs_XBC.mean()):.3}')

Xgboost, Custom_scor по фолдам  [4.46623094 4.79476934 4.68579731 5.41227752 4.24990919]
Xgboost, Custom_scor, среднее =  4.72


Результаты немного лучше, чем у RandomForest, даже удалось попасть в требуемый порог. Но это на первый взгляд, так как у нас есть результаты кросс-валидации модели RandomForest мы можем более строго оценить разницу, с точки зрения статистики


In [30]:
# используем асимптотические доверительные интервалы для разницы средних, для оценки дисперсии используем несмещенную выборочную дисперсию
# получим параметры нормального распредления разницы средних R2
mu_hat = cvs_XB.mean() - cvs_RF.mean()
sigma_hat = np.sqrt ((cvs_XB.var(ddof=1) + cvs_RF.var(ddof=1)) / cvs_XB.shape[0])
#получим границы доверительного интервала для 95% вероятностной массы распределения среднего
sc.stats.norm.interval (0.95, mu_hat, sigma_hat)

(-0.0033180304412768254, 0.022204232011159127)

In [31]:
# получим параметры нормального распредления разницы средних по Custom Loss
mu_hat_с = cvs_XBC.mean() - cvs_RFC.mean()
sigma_hat_с = np.sqrt ((cvs_XBC.var(ddof=1) + cvs_RFC.var(ddof=1)) /  cvs_XBC.shape[0])
#получим границы доверительного интервала для 95% вероятностной массы распределения среднего
sc.stats.norm.interval (0.95, mu_hat_с, sigma_hat_с)
# разница между показателями R2 лежит на границе 95% уровня значимости, но с учетом результатов сравнения Custom loss метрики
# с высокой долей вероятности можно утверждать, что разница между результатами XGBoost и RandomForest статистически значима

(0.052888721872029865, 0.9495273391966227)

Также по смыслу задачи должны хорошо сработать метрические модели, если поверхности расположены рядом, имеют похожий рейтинг и принадлежат одному и тому же селлеру, размер дисконта на них на них должен быть сопоставим

In [32]:
# тестим KNN с дефолтными параметрами 
cvs =model_selection.cross_val_score (neighbors.KNeighborsRegressor(), X, y,  cv=5, scoring='r2')
print ('KNN, R2 по фолдам ', cvs)
print (f'KNN, R2 среднее = {cvs.mean():.3}')

KNN, R2 по фолдам  [0.77844872 0.75259339 0.75011912 0.76303139 0.76084145]
KNN, R2 среднее = 0.761


In [45]:
# снова используя GridSearch (за кадром) получаем оптимальные гиперпараметры
kn =  neighbors.KNeighborsRegressor(6, p=1, weights='distance')
cvs =model_selection.cross_val_score (kn, X, y,  cv=5, scoring='r2')
print ('KNN, R2 по фолдам ', cvs)
print (f'KNN, R2 среднее = {cvs.mean():.3}')

KNN, R2 по фолдам  [0.80747658 0.80096662 0.80243617 0.80741329 0.8150226 ]
KNN, R2 среднее = 0.807


Результат не дотягивает до RandomForest и Xgboost (строгую оценку можно не проводить, результат очевиден), но в дальнейшем эту модель можно попробовать использовать при стекинге.  
Примечание: вообще данные для метрических алгоритмов рекомендуется нормировать, но в данном случае оценка **R2** ухудшается при нормировании, на мой взгляд, это происходит по двум причинам:
- некоторые числовые признаки были предварительно прологарифмированы, соответственно, масштабировались до небольших значений 
- все признаки находятся примерно в одном масштабе, зак исключением признаков Seller_Cat и геокоординат, но они, по видимому, являются ключевыми для расчета прогноза, и немасштабированные данные содержат более точную информацию о расстоянии между объектами, поэтому масштабирование и ухудшает качество прогноза


# Отбор признаков

Взглянем на то, какие признаки дают больший вклад в прогноз, а какие, возможно, вносят только шум.  
Вообще, изначально признаков в датасете очень немного и очень маловероятно, что потребуется какие-либо признаки исключать или проводить сокращения размерности признаков, но убедиться стоит

In [ ]:
# воспользуемся атрибутом feature_importances_ эстиматора у ранее обученного объекта GridSearch
#fi = pd.DataFrame (gs.best_estimator_.feature_importances_, columns=['Feature_importance'], index= X.columns.to_list()).sort_values('Feature_importance')
#fi.plot.barh(figsize=(16,10))
#plt.title ('Feature_importance, Xgboost ' );

In [ ]:
# скрин результата

<img src='./src/images/fi.png'>

Откровенно лишних признаков нет, более того, признаки с сильной корреляцией, которые были отмечены в разделе EDA оказались в топе признаков. Тем не менее, оценим как изменяется дисперсия остатков регресии, при исключении того или иного признака.

In [36]:
# делим на трейн / тест
X_train, X_test, y_train, y_test = model_selection.train_test_split ( X, y, train_size = 0.75 )

In [37]:
xbr=xgb.XGBRegressor (learning_rate=0.03, max_depth=11, n_estimators=1250, objective='reg:squarederror')
xbr.fit ( X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.03, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=11, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1250,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [38]:
# посмотрим на остатки регресcии, нарисуем границы интервала, в которое укладывается 95% отклонений
residuals = y_test - xbr.predict (X_test )
fig = plt.figure (figsize=(12,8) )
plt.hist (residuals, bins=100);
left_int = residuals.quantile (0.025)
right_int = residuals.quantile (1- 0.025)
for i in [left_int, right_int]: 
        y_max = plt.ylim()[1]
        plt.axvline(i, color="blue", linestyle='dashed', lw=2)
plt.title ('Распределение остатков регрессии' );

In [39]:
# посчитаем среднеквадратичное отклонение (Std), которое станет бенчмарком
print ('Std - ' , round (residuals.std() , 4))

Std -  0.0255


In [ ]:
# Попробуем поочередно убирать один признак и измерять Std остатков регресcии и записывать его в словарь
# Процесс занимает значительное количество времени, поэтому итоговый график вынесен отдельной картинкой

In [ ]:
#res={}
#res['bench'] = residuals.std()
#xbr1=xgb.XGBRegressor (learning_rate=0.03, max_depth=11, n_estimators=1250)
#for c in X.columns.to_list():
#  cl= X.columns.to_list()
#  cl.remove(c)
#  xbr1.fit (X_train.loc[:, cl], y_train)  
#  diff = y_test - xbr1.predict (X_test.loc[:, cl])
#  y_pred = xbr1.predict (X_test.loc[:, cl])
#  res[c] = diff.std()
#fe = pd.DataFrame.from_dict (res, orient='index' , columns=['Std']).sort_values ('Std')

In [ ]:
#fig = plt.figure (figsize=(12,8) )
#barviz = plt.barh ( fe.index , fe['Std'])
#barviz[17].set_color('b')
#plt.title ('Std остатков регрессии, в зависимости от убранного признака ' );

<img src='./src/images/fe.png'>

Попробуем убрать 3 признака, лежащих ниже бенчмарка: **OperatorR1** и **CityType**, которые мы отметили в разделе EDA и **GRP**, который находится внизу списка

In [40]:
X_trunc = X.drop (['GRP', 'OperatorR1' , 'CityType' ] , axis=1)

In [41]:
cvs = model_selection.cross_val_score (xgb.XGBRegressor ( learning_rate=0.03, max_depth=11, n_estimators=1250, objective='reg:squarederror' ),
                                       X_trunc, y,  cv=5, scoring='r2')
print ('Xgboost_trunc, R2 по фолдам ', cvs)
print (f'Xgboost_trunc, R2 среднее = {cvs.mean():.4}')

Xgboost, R2 по фолдам  [0.90302196 0.88455736 0.8824536  0.88376288 0.9057206 ]
Xgboost, R2 среднее = 0.8919


In [42]:
cvs = model_selection.cross_val_score (xgb.XGBRegressor ( learning_rate=0.03, max_depth=11, n_estimators=1250 , objective='reg:squarederror'),
                                       X_trunc, y,  cv=5, scoring=custom_scorer)
print ('Xgboost_trunc, Custom_scor, по фолдам ', np.negative (cvs) )
print (f'Xgboost_trunc, Custom_scor, среднее =  {np.negative (cvs.mean()):.4}')

Xgboost, Custom_scor, по фолдам  [4.3936093  4.90374137 4.61314929 5.0127134  4.06828914]
Xgboost, Custom_scor, среднее =  4.598


Есть небольшое улучшение работы модели, интересно, что признак **GRP** (рейтинг конструкции), который по смыслу должен был быть одним из основных влияющих на дисконт, в итоге оказался бесполезным для прогноза

В заключение попробуем использовать стекинг, объединив предсказания Xgboost и KNN, возможно они дополнят друг друга

In [ ]:
est_list = [('xbr' , xbr), ('KNN',kn) ]
stc = ensemble.StackingRegressor(est_list, final_estimator=ensemble.RandomForestRegressor ( ) , cv=5)
stc.fit (X_train, y_train)
print (f'Stacking, R2  =  {metrics.r2_score (y_test, stc.predict (X_test)):.4}')
print (f'Stacking, custom score  =  {custom_loss (y_test, stc.predict (X_test)):.4}')

Улучшения не произошло, получилось обычное усреднение результатов прогнозов двух моделей

# Итоги

С помощью модели XGBoost удалось попасть в нужный интервал, но не очень далеко от правой границы интервала, и на новых данных можно легко выпасть за его пределы, необходимо более подробное изучение признаков